In [154]:
import sys
sys.path.append('../SQL')
from urllib import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
import numpy as np
import time
import datetime
import MySQLdb

In [155]:
# call selenium and chromedriver to get driver
def driveChrome(url):
    driver = webdriver.Chrome('./chromedriver') # CHANGE THE DRIVER TYPE and PATH HERE!!

    # mimic mouse operation to scroll down and show the full page
    driver.get(url)
    driver.execute_script("window.scrollBy(0,3000)")
    time.sleep(0.1)
    driver.execute_script("window.scrollBy(0,5000)")
    time.sleep(0.1)
    driver.execute_script("window.scrollBy(0,5000)")
    time.sleep(0.1)
    driver.execute_script("window.scrollBy(0,5000)")
    time.sleep(0.1)
    driver.execute_script("window.scrollBy(0,5000)")
    time.sleep(0.1)
    
    return driver

In [156]:
# resolve html to soup
def getSoup(url, driver):
    response = urlopen(url)
    html = response.read()
    soup = BeautifulSoup(driver.page_source, "lxml")
    return soup;

In [157]:
# transfer month into number
def transMon(day):
    monf = day.text.split(" "); # monf[0] - Jan - Dec    
    #Jan 04, 2018
    mon = "00"
    if monf[0] == "Jan":
        mon = "01"
    elif monf[0] == "Feb":
        mon = "02"
    elif monf[0] == "Mar":
        mon = "03"            
    elif monf[0] == "Apr":
        mon = "04"            
    elif monf[0] == "May":
        mon = "05"            
    elif monf[0] == "Jun":
        mon = "06"            
    elif monf[0] == "Jul":
        mon = "07"            
    elif monf[0] == "Aug":
        mon = "08"
    elif monf[0] == "Sep":
        mon = "09"        
    elif monf[0] == "Oct":
        mon = "10"
    elif monf[0] == "Nov":
        mon = "11"            
    elif monf[0] == "Dec":
        mon = "12"     
        
    dayf = monf[1].strip(',')
    resdate = monf[2]+"-"+mon+"-"+dayf
    return resdate  

In [158]:
def getHistoricalData(stockCode):
    url = getURL(stockCode)
    driver  = driveChrome(url)
    soup = getSoup(url,driver)
    # quit selenium driver
    driver.quit()
    
    # get dates
    days = soup.find_all("td", class_ = "Py(10px) Ta(start) Pend(10px)") # <td class="Py(10px) Ta(start) Pend(10px)"><span>Feb 28, 2018</span></td>
    dates = []        
    for day in days:
        day = transMon(day)
        #print(day)
        dates.append(day)
    
    
    # delete invalid date
    bad = []
    tr = soup.find_all("tr", class_ = "BdT Bdc($c-fuji-grey-c) Ta(end) Fz(s) Whs(nw)")

    for inner in tr:
        badi = inner.find("td", class_ = "Ta(c) Py(10px) Pstart(10px)")
        if badi:
            #print(bad.text)
            badparent = badi.find_parent()
            baddate = badparent.find("td", class_ = "Py(10px) Ta(start) Pend(10px)")
            bad.append(transMon(baddate))

    for baditem in bad:
        dates.remove(baditem)
    
    # get prices and volume
    elements = soup.find_all("td", class_ = "Py(10px) Pstart(10px)") # <td class="Py(10px) Pstart(10px)"><span>1,123.03</span></td>

    openp = [] # open price
    highp = [] # high price
    lowp = [] # low price
    closep = [] # close price
    adjclosep = [] # adjusted close price
    volume = [] # volume
    
    
    counter = 0
    for element in elements:
        #print(element.text)
        text = element.text.replace("," , "")
        counter+=1;
        if counter % 6 == 1:
            openp.append(float(text))
        elif counter % 6 == 2:
            highp.append(float(text))
        elif counter % 6 == 3:
            lowp.append(float(text))
        elif counter % 6 == 4:
            closep.append(float(text))
        elif counter % 6 == 5:
            adjclosep.append(float(text))
        elif counter % 6 == 0:
            volume.append(int(text))
     
    # generate symbol column
    num = len(volume)
    symbols = []
    for i in range(0,num):
        symbols.append(stockCode)        
    
    print(len(symbols), len(dates), len(openp), len(highp), len(lowp), len(closep), len(adjclosep), len(volume))
    tmp = [symbols,dates, openp, highp, lowp, closep, adjclosep, volume]
    res = np.stack(tmp,axis = 1)
      
    # insert into database
    for record in res:
        dbInsertHistoryTime(record)
    
    return res

In [159]:
# generate Yahoo Finance url
def getURL(stockCode):
    return "https://finance.yahoo.com/quote/"+ stockCode + "/history?p=" + stockCode

In [160]:
# insert data into database
def dbInsertHistoryTime(list):
    db = MySQLdb.connect("127.0.0.1", "root", "passwd", "stockDB") # Change your database info here!
    cursor = db.cursor()
    sql = "INSERT INTO History_Time_Data(symbol, stock_date, open_price, high_price, low_price, close_price, adj_close, volume) VALUES(" + "'"+list[0] + "'," + "'" + list[1] + "'" + "," + list[2] + "," + list[3] + "," + list[4] + "," + list[5] + "," + list[6] + "," + list[7] + ")" + ";"
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()
        print("Insert historical data Failed!")
    db.close()

------------------------------- ** Examples ** --------------------------------------

In [161]:
"""
Example: example = getHistoricalData("yourStockCode")


example: records of all year
example[x]: record of day x from today

example[x][0]: date (String)
example[x][1]: open price (float)
example[x][2]: high price (float)
example[x][3]: low price (float)
example[x][4]: close price* (float)
example[x][5]: adjusted close price** (float)
example[x][6]: volume (int)

Index from 0 to 251(252 total).
0   is the most recent day(Feb.28.2018)
251 is the least recent day(Mar.01.2017)
Tested Feb.28.2018 by Xin Yang.

*Close price adjusted for splits. **Adjusted close price adjusted for both dividends and splits.
"""

'\nExample: example = getHistoricalData("yourStockCode")\n\n\nexample: records of all year\nexample[x]: record of day x from today\n\nexample[x][0]: date (String)\nexample[x][1]: open price (float)\nexample[x][2]: high price (float)\nexample[x][3]: low price (float)\nexample[x][4]: close price* (float)\nexample[x][5]: adjusted close price** (float)\nexample[x][6]: volume (int)\n\nIndex from 0 to 251(252 total).\n0   is the most recent day(Feb.28.2018)\n251 is the least recent day(Mar.01.2017)\nTested Feb.28.2018 by Xin Yang.\n\n*Close price adjusted for splits. **Adjusted close price adjusted for both dividends and splits.\n'

In [162]:
getHistoricalData("GOOG")    

(251, 251, 251, 251, 251, 251, 251, 251)


array([[u'GOOG', u'2018-03-01', u'1107.87', ..., u'1069.52', u'1069.52',
        u'2452400'],
       [u'GOOG', u'2018-02-28', u'1123.03', ..., u'1104.73', u'1104.73',
        u'1882600'],
       [u'GOOG', u'2018-02-27', u'1141.24', ..., u'1118.29', u'1118.29',
        u'1774100'],
       ..., 
       [u'GOOG', u'2017-03-07', u'827.4', ..., u'831.91', u'831.91',
        u'1037600'],
       [u'GOOG', u'2017-03-06', u'826.95', ..., u'827.78', u'827.78',
        u'1109000'],
       [u'GOOG', u'2017-03-03', u'830.56', ..., u'829.08', u'829.08',
        u'896400']],
      dtype='<U32')

In [163]:
getHistoricalData("AABA")

(251, 251, 251, 251, 251, 251, 251, 251)


array([[u'AABA', u'2018-03-01', u'75.04', ..., u'73.74', u'73.74',
        u'9719900'],
       [u'AABA', u'2018-02-28', u'75.91', ..., u'74.85', u'74.85',
        u'8566500'],
       [u'AABA', u'2018-02-27', u'77.86', ..., u'76.13', u'76.13',
        u'12425700'],
       ..., 
       [u'AABA', u'2017-03-07', u'45.5', ..., u'45.73', u'45.73',
        u'5116800'],
       [u'AABA', u'2017-03-06', u'45.61', ..., u'45.64', u'45.64',
        u'6694900'],
       [u'AABA', u'2017-03-03', u'45.88', ..., u'45.91', u'45.91',
        u'6758300']],
      dtype='<U32')

In [164]:
getHistoricalData("CSCO")

(251, 251, 251, 251, 251, 251, 251, 251)


array([[u'CSCO', u'2018-03-01', u'44.68', ..., u'43.8', u'43.8',
        u'32002200'],
       [u'CSCO', u'2018-02-28', u'45.27', ..., u'44.78', u'44.78',
        u'26885200'],
       [u'CSCO', u'2018-02-27', u'45.41', ..., u'45.04', u'45.04',
        u'27419300'],
       ..., 
       [u'CSCO', u'2017-03-07', u'34.0', ..., u'34.2', u'33.06',
        u'21033700'],
       [u'CSCO', u'2017-03-06', u'34.04', ..., u'34.19', u'33.05',
        u'17605100'],
       [u'CSCO', u'2017-03-03', u'34.42', ..., u'34.29', u'33.14',
        u'14175000']],
      dtype='<U32')

In [165]:
getHistoricalData("T")

(251, 251, 251, 251, 251, 251, 251, 251)


array([[u'T', u'2018-03-01', u'36.31', ..., u'36.0', u'36.0', u'28413500'],
       [u'T', u'2018-02-28', u'37.18', ..., u'36.3', u'36.3', u'29281700'],
       [u'T', u'2018-02-27', u'37.42', ..., u'36.87', u'36.87', u'21420600'],
       ..., 
       [u'T', u'2017-03-07', u'41.9', ..., u'41.88', u'39.82', u'16250900'],
       [u'T', u'2017-03-06', u'41.83', ..., u'41.96', u'39.9', u'14177000'],
       [u'T', u'2017-03-03', u'42.05', ..., u'42.01', u'39.94', u'17179400']],
      dtype='<U32')

In [166]:
getHistoricalData("WMT")

(251, 251, 251, 251, 251, 251, 251, 251)


array([[u'WMT', u'2018-03-01', u'90.17', ..., u'89.08', u'89.08',
        u'18680300'],
       [u'WMT', u'2018-02-28', u'92.0', ..., u'90.01', u'90.01',
        u'14143700'],
       [u'WMT', u'2018-02-27', u'93.63', ..., u'91.52', u'91.52',
        u'13376100'],
       ..., 
       [u'WMT', u'2017-03-07', u'69.9', ..., u'69.87', u'68.11', u'7282900'],
       [u'WMT', u'2017-03-06', u'70.0', ..., u'69.88', u'68.12', u'9182600'],
       [u'WMT', u'2017-03-03', u'70.76', ..., u'70.03', u'68.27',
        u'9297900']],
      dtype='<U32')

In [167]:
getHistoricalData("NOK")

(251, 251, 251, 251, 251, 251, 251, 251)


array([[u'NOK', u'2018-03-01', u'5.77', ..., u'5.76', u'5.76', u'16202200'],
       [u'NOK', u'2018-02-28', u'5.88', ..., u'5.8', u'5.8', u'11084300'],
       [u'NOK', u'2018-02-27', u'5.91', ..., u'5.87', u'5.87', u'17203900'],
       ..., 
       [u'NOK', u'2017-03-07', u'5.23', ..., u'5.23', u'5.08', u'8366700'],
       [u'NOK', u'2017-03-06', u'5.29', ..., u'5.28', u'5.13', u'7381800'],
       [u'NOK', u'2017-03-03', u'5.26', ..., u'5.28', u'5.13', u'11993000']],
      dtype='<U32')

In [168]:
getHistoricalData("NFLX")

(251, 251, 251, 251, 251, 251, 251, 251)


array([[u'NFLX', u'2018-03-01', u'292.75', ..., u'290.39', u'290.39',
        u'11816300'],
       [u'NFLX', u'2018-02-28', u'293.1', ..., u'291.38', u'291.38',
        u'7653500'],
       [u'NFLX', u'2018-02-27', u'294.77', ..., u'290.61', u'290.61',
        u'9416500'],
       ..., 
       [u'NFLX', u'2017-03-07', u'141.99', ..., u'141.43', u'141.43',
        u'4241000'],
       [u'NFLX', u'2017-03-06', u'141.0', ..., u'141.94', u'141.94',
        u'7116100'],
       [u'NFLX', u'2017-03-03', u'139.69', ..., u'139.14', u'139.14',
        u'5399800']],
      dtype='<U32')

In [169]:
getHistoricalData("APA")

(251, 251, 251, 251, 251, 251, 251, 251)


array([[u'APA', u'2018-03-01', u'34.24', ..., u'34.27', u'34.27',
        u'5859300'],
       [u'APA', u'2018-02-28', u'35.0', ..., u'34.15', u'34.15', u'8278800'],
       [u'APA', u'2018-02-27', u'35.89', ..., u'34.8', u'34.8', u'5756600'],
       ..., 
       [u'APA', u'2017-03-07', u'52.37', ..., u'51.28', u'50.19',
        u'3375300'],
       [u'APA', u'2017-03-06', u'52.07', ..., u'52.19', u'51.08',
        u'2125400'],
       [u'APA', u'2017-03-03', u'52.02', ..., u'52.09', u'50.98',
        u'2856900']],
      dtype='<U32')

In [170]:
getHistoricalData("NKE")

(251, 251, 251, 251, 251, 251, 251, 251)


array([[u'NKE', u'2018-03-01', u'67.24', ..., u'66.32', u'66.12',
        u'7262800'],
       [u'NKE', u'2018-02-28', u'68.1', ..., u'67.03', u'66.83', u'6091800'],
       [u'NKE', u'2018-02-27', u'70.0', ..., u'68.03', u'67.82', u'6843100'],
       ..., 
       [u'NKE', u'2017-03-07', u'56.71', ..., u'56.55', u'55.81',
        u'7124400'],
       [u'NKE', u'2017-03-06', u'56.7', ..., u'56.77', u'56.03', u'6896800'],
       [u'NKE', u'2017-03-03', u'57.8', ..., u'56.7', u'55.96', u'9181600']],
      dtype='<U32')

In [171]:
getHistoricalData("GE")

(251, 251, 251, 251, 251, 251, 251, 251)


array([[u'GE', u'2018-03-01', u'14.16', ..., u'14.02', u'14.02',
        u'90084900'],
       [u'GE', u'2018-02-28', u'14.6', ..., u'14.11', u'14.11', u'88614100'],
       [u'GE', u'2018-02-27', u'14.67', ..., u'14.5', u'14.5', u'93024000'],
       ..., 
       [u'GE', u'2017-03-07', u'29.93', ..., u'29.86', u'28.88',
        u'29982900'],
       [u'GE', u'2017-03-06', u'30.0', ..., u'30.0', u'29.01', u'23420600'],
       [u'GE', u'2017-03-03', u'30.19', ..., u'30.12', u'29.13',
        u'19957000']],
      dtype='<U32')